# Import

In [1]:
import pandas as pd
import json
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
master_port = pd.read_csv('ports_master.csv').iloc[:,1:]
# port = master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r']].itertuples(index=False)
PORTS = list(master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu']].itertuples(index=False, name=None))
# Port = namedtuple('port',['LOCODE','draft0','draft1','draft2','draft3','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu'])
# PORT[]
PORTS [55:61],len(PORTS)
ves_size = [170,275,335,390]
regions_cod = ('AFR','EUR','ELT','WLT','MDE','NAM','OCE')
for reg in regions_cod:
    sc = (reg, 20000, 20000, 20000, 20000, 0, 0, 0, 0, 0, 999)
    PORTS.append(sc)

In [3]:
pv = [(p,v) for p in range(len(PORTS)) for v in range(4) ]

## Fix transshipment

In [4]:
change_tran = lambda x : x.replace('trans','port_trans') if 'trans' == x[:5] == 'trans' else x
def check_tran(path):
    with open(path, "r") as json_file:
        sol1 = json.load(json_file)
    res = sol1['CPLEXSolution']['variables']
    res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'],)[['name','value']]
    obj = int(float(sol1['CPLEXSolution']['header']['objectiveValue']))
    res_df['value'] = res_df['value'].astype(float)
    ptran_chk = [n for n in res_df.name.tolist() if 'port_trans' in n ]
    tran_chk = [n for n in res_df.name.tolist() if 'trans' in n[:5] ]
    print(path,len(tran_chk),len(ptran_chk))
    var = []
    for r in res_df.reset_index().itertuples():
        n = change_tran(r.name)
        var.append({'name':n,'value':int(r.value)})
    return var,obj


In [6]:
def fix_transship(dirpath = '.'):
    for j in [f for f in os.listdir('.') if f[-4:]=='json']:
        var,obj = check_tran(j)
        new_dict = {'CPLEXSolution': {'version': '1.0',
            'header': {'problemName': 'Container_network_3.2_2015',
            'objectiveValue': obj,
            'solved_by': 'cplex_local'},
            'variables': var}
            }
        print(obj)
        with open(j, "w") as out_file:
            json.dump(new_dict, out_file, indent = 6)     
        print(f'{j} Done')

fix_transship()

Solution_2015_02151851.json 48 0
444347408
Solution_2015_02151851.json Done
Solution_2015_02151947.json 48 0
444347408
Solution_2015_02151947.json Done
Solution_2015_02160026.json 0 48
444347408
Solution_2015_02160026.json Done
Solution_2015_02161042.json 0 48
444347408
Solution_2015_02161042.json Done
Solution_2015_02161410.json 0 48
444347408
Solution_2015_02161410.json Done
Solution_2015_02161855.json 0 48
444347408
Solution_2015_02161855.json Done
Solution_2015_02170028.json 0 48
444347408
Solution_2015_02170028.json Done
Solution_2015_02181736.json 0 48
444347408
Solution_2015_02181736.json Done


# Util

In [11]:
def import_json_sol(jsonpath):
    with open(jsonpath) as json_file:
        sol1 =json.load(json_file)
    res = len(sol1['CPLEXSolution']['variables'])
    obj = int(float(sol1['CPLEXSolution']['header']['objectiveValue']))
    print(f"Objective Value {obj}, \tVaraibles {res}")
    res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'],)[['name','value']]
    res_df['value'] = res_df['value'].astype(float)
    # lst.name
    return obj,res_df

In [17]:
def get_berth_occupancy(df,verbose = False):
    vessel_len = [175, 275,335,390]
    oc_rate = {}
    val_all = df.name.tolist()
    lst = {(p,v):s for p in range(len(PORTS)) for v in range(4) for s in val_all if f'port_calls_{p}_{v}' in s}
    if verbose : print(lst.keys())
    for p in range(len(PORTS)):
        port_berth = PORTS[p][1:5]
        port_call = []
        for v in range(4):
            # if p==15 :print("15",lst(p,v) in lst)
            if (p,v) in lst.keys():
                port_call.append(df[df.name == lst[(p,v)]].value.sum())
            else:
                port_call.append(0)
        port_used = [a * b for a, b in zip(port_call,ves_size)]
        port_oc = sum(port_used) / sum(port_berth)
        if (port_oc==0 or port_oc>7) and verbose:
            print(p,port_oc,PORTS[p][0],port_used,port_call,port_berth )
        oc_rate[p] = (PORTS[p][0],port_oc)
    return oc_rate
# berth_occupancy(res_df)


In [21]:
obj,res_df = import_json_sol('Solution_2015_02170028.json ')
oc = get_berth_occupancy(res_df)
pd.DataFrame(oc).T

Objective Value 444347408, 	Varaibles 5354


,0,1
0,BDCGP,0.444444
1,CNDCB,0
2,CNYTG,0
3,CNQZH,0.435897
4,CNFOC,1.11548
...,...,...
69,ELT,0.0933125
70,WLT,0.0668125
71,MDE,0.731
72,NAM,0.472062


In [19]:
def get_throughput(df,annual=False,toint = True):
    """
    Throughput only
    """
    val_all = df.name.tolist()
    lst = [s for s in val_all if 'port' in s]
    _df = df[res_df.name.isin(lst)].copy()
    print(_df.shape)
    _df['Port_ind']= _df.name.apply(lambda x: x[-2:].replace('_',''))
    # thrp_all['Port_ind'] = thrp_all['Port_ind'].replace('_','')
    _df['Port_name']= _df.Port_ind.apply(lambda x: PORTS[int(x)][0])
    _df = _df.sort_values(by='value',ascending=False)
    if toint:
        _df.value = _df.value.astype(int)
    _df = _df[['Port_ind','Port_name','value']]
    if annual:
        _df['Annually'] = _df.value.apply(lambda x: x*50/1e6)
    return _df.reset_index(drop=True)

In [26]:
def get_ope_param(df,param):
    """
    ope params included
        port_load, port_unload,
        imp,exp,port_trans
    """
    valid_name = ['imp','exp', 'port_load', 'port_unload', 'trans']
    if param not in valid_name:
        print('Invalid Parameter')
        return None

    val_all = df.name.tolist()
    lst = [s for s in val_all if param in s]
    print(len(lst))
    _df = df[res_df.name.isin(lst)].copy()
    print(_df.shape)
    _df['Port_ind']= _df.name.apply(lambda x: x.split('_')[-2])
    return _df


# Integer Read
 => Throughput

 => Transhipment

 => Occupancy

In [27]:
obj, res_df = import_json_sol('Solution_2015_02151851.json')
res_df

Objective Value 444347408, 	Varaibles 1


KeyError: "None of [Index(['name', 'value'], dtype='object')] are in the [columns]"

In [29]:
res_df[res_df.name.str[:5]=='trans'].name

,name,value
5258,trans_0_48_5,3786.0
5259,trans_0_60_13,313.0
5260,trans_1_42_6,348.0
5261,trans_1_48_5,696.0
5262,trans_1_62_8,307.0
5263,trans_4_42_6,119.0
5264,trans_5_42_6,174.0
5265,trans_6_48_5,787.0
5266,trans_7_42_6,566.0
5267,trans_7_48_5,7519.0


In [17]:
get_ope_param(res_df,'Port_trans')

Invalid Parameter


In [16]:
# [s for s in val_all if 'ves_deploy' in s]
val_all = [n.split('_')[0] for n in res_df.name.tolist()]

In [58]:
get_throughput('ves_deploy',res_df)

NameError: name 'get_throughput' is not defined

In [ ]:
get_all_portval('ves_deploy',res_df)

In [ ]:
val_all = res_df.name.tolist()
sel_val = [s for s in val_all if 'relax_route_cap' in s]
cap_relax = res_df[res_df.name.isin(sel_val)]
cap_relax.sort_values(by='value',ascending=False)

In [ ]:
with open('Solution_relaxed.json') as json_file:
    sol1 =json.load(json_file)
print(len(sol1),type(sol1))
res = len(sol1['CPLEXSolution']['variables'])
res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'],)[['name','value']]
res_df['value'] = res_df['value'].astype(float)
res_df

In [ ]:
CHNBG SGSIN CNSHG MYPKG KRPUS HKHKG CHYTN IDJKT CNSHK XM